In [1]:
#pip install langgraph

In [2]:
#pip install optuna

In [3]:
import joblib
import pandas as pd
from typing import TypedDict
from langgraph.graph import StateGraph
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from groq import Groq
import optuna

In [4]:
# === Load & preprocess the breast cancer dataset ===
df = pd.read_csv("Breast_Cancer.csv")
df.drop(columns=["Unnamed: 32", "id"], inplace=True)
df["diagnosis"] = df["diagnosis"].map({"M": 1, "B": 0})

X = df.drop("diagnosis", axis=1)
y = df["diagnosis"]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

In [5]:
# Save the scaler (used after retraining too)
joblib.dump(scaler, "scaler_1.pkl")

# === Load or train initial model ===
model = joblib.load("svc_breast_cancer_model.pkl")
y_pred = model.predict(X_test)
MODEL_ACCURACY = accuracy_score(y_test, y_pred)

In [6]:
# === Groq Setup ===
client = Groq(api_key="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")  # replace with your key

# === Define LangGraph State ===
class MLOpsState(TypedDict):
    status: str
    accuracy: float

In [7]:
# === Node 1: Monitor model performance ===
def monitor_model(state: MLOpsState):
    print(f"Model Accuracy: {state['accuracy']}")
    return {"status": "drift_detected" if state["accuracy"] < 0.90 else "model_healthy", "accuracy": state["accuracy"]}


In [8]:
# === Node 2: Ask Groq LLM if retraining is needed ===
def ask_llm_to_retrain(state: MLOpsState):
    prompt = "The model accuracy has dropped below acceptable limits. Should I retrain the model?"
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-70b-8192"
    )
    reply = response.choices[0].message.content.lower()
    print("LLM says:", reply)
    return {"status": "retrain" if "yes" in reply else "done", "accuracy": state["accuracy"]}


In [9]:
# === Node 3: Retrain the model ===

def objective(trial):
    C = trial.suggest_loguniform("C", 1e-3, 1e3)
    gamma = trial.suggest_loguniform("gamma", 1e-4, 1e1)

    model = SVC(probability=True, C=C, gamma=gamma)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return accuracy_score(y_test, preds)

def retrain_model(state: MLOpsState):
    print("Starting hyperparameter tuning with Optuna...")

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)

    print("Best parameters:", study.best_params)

    # Retrain with best parameters
    best_model = SVC(probability=True, **study.best_params)
    best_model.fit(X_train, y_train)

    new_accuracy = accuracy_score(y_test, best_model.predict(X_test))
    joblib.dump(best_model, "svc_breast_cancer_model.pkl")

    print(f"Retrained model accuracy: {new_accuracy}")
    return {"status": "deploy", "accuracy": new_accuracy}


In [10]:
# === Node 4: Simulated deployment ===
def deploy_model(state: MLOpsState):
    print(f"Deploying model with accuracy: {state['accuracy']}")
    return {"status": "done", "accuracy": state["accuracy"]}

In [11]:
# === Node 5: End ===
def done(state: MLOpsState):
    print("MLOps Workflow Complete!")
    return state

In [12]:
# === Build the LangGraph ===
workflow = StateGraph(MLOpsState)
workflow.add_node("monitor", monitor_model)
workflow.add_node("decision", ask_llm_to_retrain)
workflow.add_node("retrain", retrain_model)
workflow.add_node("deploy", deploy_model)
workflow.add_node("done", done)

workflow.set_entry_point("monitor")
workflow.add_conditional_edges("monitor", lambda s: "decision" if s["status"] == "drift_detected" else "done")
workflow.add_conditional_edges("decision", lambda s: "retrain" if s["status"] == "retrain" else "done")
workflow.add_edge("retrain", "deploy")
workflow.add_edge("deploy", "done")

agent = workflow.compile()

In [13]:
# === Run the Workflow ===
output = agent.invoke({"status": "monitoring", "accuracy": MODEL_ACCURACY})
print("Final Output:", output)

Model Accuracy: 0.9824561403508771
MLOps Workflow Complete!
Final Output: {'status': 'model_healthy', 'accuracy': 0.9824561403508771}
